In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
pip install tiktoken

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 10.7 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires cohere, which is not installed.
llmx 0.0.15a0 requires openai, which is not installed.


In [ ]:
import tiktoken
import os
import time
from bs4 import BeautifulSoup
import pprint

In [ ]:
!pip install git+https://github.com/huggingface/transformers.git@refs/pull/25740/head accelerate

!pip install transformers
from transformers import AutoTokenizer
import transformers
import torch
#import accelerate

  Cloning https://github.com/huggingface/transformers.git (to revision refs/pull/25740/head) to /tmp/pip-req-build-72o8a0r2
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers.git /tmp/pip-req-build-72o8a0r2
  Running command git fetch -q https://github.com/huggingface/transformers.git refs/pull/25740/head
  Running command git checkout -q c6c6daa3f07e753cff91a08c4294df4a6ea6227b
  Resolved https://github.com/huggingface/transformers.git to commit c6c6daa3f07e753cff91a08c4294df4a6ea6227b
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 258.1/258.1 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 25.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 49.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 77.2 MB/s eta 0:00:00
  Cre

In [ ]:
model="codellama/CodeLlama-13b-Instruct-hf"
tokenizer = AutoTokenizer.from_pretrained(model)
pipeline = transformers.pipeline(
    "text-generation",
    model=model,
    torch_dtype=torch.float16,
    device_map="auto",
    #low_cpu_mem_usage=False
)

(…)ct-hf/resolve/main/tokenizer_config.json:   0%|          | 0.00/749 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

(…)-Instruct-hf/resolve/main/tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

(…)-hf/resolve/main/special_tokens_map.json:   0%|          | 0.00/411 [00:00<?, ?B/s]

(…)13b-Instruct-hf/resolve/main/config.json:   0%|          | 0.00/589 [00:00<?, ?B/s]

...

model-00001-of-00003.safetensors:   0%|          | 0.00/9.95G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/9.90G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/6.18G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

(…)t-hf/resolve/main/generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

In [ ]:
# def llama_completion(prompt,temperature_new=0.05, top_p_new=1):
#     sequences = pipeline(
#       prompt,
#       do_sample=True,
#       top_k=10, # top outputs
#       temperature=temperature_new,
#       top_p=top_p_new,
#       num_return_sequences=1,
#       eos_token_id=tokenizer.eos_token_id,
#       max_length=200,
#       add_special_tokens=False)
#     return sequences

def llama_completion(prompt):
  sequences = pipeline(
      prompt,
      do_sample=True,
      top_k=10,
      num_return_sequences=1,
      eos_token_id=tokenizer.eos_token_id,
      max_length=200,
  )
  return sequences

In [ ]:
directory = '/content/drive/MyDrive/Colab Notebooks/data/333-01.xml'

In [ ]:
# #####parsing example
# import xml.etree.ElementTree as ET

# # Read the XML file
# with open(directory, 'r') as file:
#     xml_content = file.read()

# # Parse XML content
# root = ET.fromstring(xml_content)

# # Extract the CDATA text
# text_element = root.find('TEXT')
# cdata_text = text_element.text


In [ ]:
list_of_text_contents = []
list_of_files = []
directory='/content/drive/MyDrive/Colab Notebooks/data'
for filename in os.listdir(directory):
    f = os.path.join(directory, filename)
    if os.path.isfile(f):
        print(os.path.basename(os.path.normpath(f))[:-4])
        list_of_files.append(os.path.basename(os.path.normpath(f))[:-4])
        with open(f) as fp:
            soup = BeautifulSoup(fp, features="xml")
            text = soup.find('TEXT')
            text_content = text.contents[0]
            list_of_text_contents.append(text_content)

for i in range(len(list_of_text_contents)):
    sys = "You are a professional physician that can identify patient's record and private information that cannot be leaked."
    user = "From the following text: " + list_of_text_contents[i] + "Find patient name, DOB, Sex."
    prompt = f"<s>[INST]<<SYS>>{sys}<</SYS>>/n[INST]User:{user}[/INST]</s>"

    sequences = llama_completion(prompt)
    for seq in sequences:
      rewrite_finding = seq['generated_text']
      print(f"Result: {seq['generated_text']}")

    # rewrite_file = list_of_files[i] + "_anonymized.txt"

    # with open(rewrite_file, "w") as f:
    #     f.write(rewrite_finding)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


333-01


/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1268: UserWarning: Input length of input_ids is 2876, but `max_length` is set to 200. This can lead to unexpected behavior. You should consider increasing `max_new_tokens`.
  warnings.warn(


Result: <s>[INST]<<SYS>>You are a professional physician that can identify patient's record and private information that cannot be leaked.<</SYS>>/n[INST]User:From the following text: 



Record date: 2122-02-23


**The source of this note is the MCH Emergency Dept.
Information System. All updates should originate in EDIS.**

Memorial Community HospitalEmergency Department Record
General Note &nbsp       &nbsp    &nbsp       &nbsp    &nbsp       &nbsp    &nbsp       &nbsp   EDIS Note Status: Signed
Patient: Edgar, Veda &nbsp       &nbsp   
MRN: 9262764        
DOB:  6/4/2059     
Sex: F
Registration Date/Time: 2/23/2122 21:17
General ED Note
Note Status: Signed
ED Dismissal Date Time:       02/24/22 18:58
This note has been electronically signed by Vincent Fournier, MD-Attending 02/24/22 21:26
Clinical staff documenting in the ED note include:
  Vincent Fournier, MD-Attending [vmf47]
  Xion Eubanks, MD [xie42]

Date/Time of Encounter:02/23/22 21:25 [xie42]

Chief Complaint:SOB [xie42]
